In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  !mkdir src
  !cp -r drive/MyDrive/IFT780/HappyWhale/src/* src/
  !mkdir data
  !cp -r drive/MyDrive/IFT780/HappyWhale/data/* data/
  IN_COLAB = True
  #!pip install timm
except:
  IN_COLAB = False
  #%pip install -r requirements.txt

# Pour automatiquement recharger les modules externes
# voir http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# HappyWhale Challenge - Team WhalePlayed

This notebook presents the work done by Gaétan Rey, Julien Levarlet and Timothée Wright, as part of the challenge https://www.kaggle.com/competitions/happy-whale-and-dolphin/overview .

Imports :

In [2]:
import torch.nn as nn
import torch
from src.ModelTrainTestManager import ModelTrainTestManager, optimizer_setup
from src.DataManager import DataManager
from src.Models.ResNet import ResNet
from src.Models.HappyWhaleModel import HappyWhaleModel

Parameters for the data :

In [3]:
data_csv = "data/common_train.csv"
dataFolderPath = "data/common_cropped_train_imgs"

Parameters for the training :

In [4]:
batch_size = 10
learning_rate = 0.01
optimizer_factory = optimizer_setup(torch.optim.Adam, lr=learning_rate)

test_percentage = 0.2
val_percentage = 0.2

exp_name = "HappyWhale"

Parameters for the model :

In [5]:
num_classes=21
in_channels=3
depth=4
option="small"
size=256

# ArcFace Hyperparameters
arcFace_config = {
    "s": 30.0,  # scale (The scale parameter changes the shape of the logits. The higher the scale, the more peaky the logits vector becomes.)
    "m": 0.50,  # margin (margin results in a bigger separation of classes in your training set)
    "ls_eps": 0.0,
    "easy_margin": False
}

model = HappyWhaleModel("tf_efficientnet_b0_ns", 512, num_class=20, arcface_config=arcFace_config)
#ResNet(num_classes, in_channels, depth, option, size, use_arcface=False)

1280


In [6]:
data_manager = DataManager(data_csv, dataFolderPath, batch_size,
                test_percentage, val_percentage, verbose=True)

model_trainer = ModelTrainTestManager(model=model,
                                        data_manager = data_manager,
                                        loss_fn=nn.CrossEntropyLoss(),
                                        optimizer_factory=optimizer_factory,
                                        exp_name = exp_name ,
                                        learning_rate=learning_rate,
                                        use_cuda=True)

                image  individual_id
0  000a8f2d5c316a.jpg             12
1  001001f099519f.jpg              2
2  00144776eb476d.jpg             12
3  0024057bbc89a4.jpg              1
4  0028f6fa123686.jpg              9
Dataset size : 1000
Size of validation set : 160
Size of test set : 200
Size of train set : 640


In [7]:
epoch=10
model_trainer.train(epoch)

  0%|          | 0/64 [00:00<?, ?it/s]

Epoch: 1 of 10


  0%|          | 0/64 [00:00<?, ?it/s]

Validation loss 17.411
Epoch: 2 of 10


 84%|████████▍ | 54/64 [00:11<00:02,  4.66it/s, loss=14.569]


KeyboardInterrupt: 

In [ ]:
model_trainer.evaluate_on_test_set()
model_trainer.plot_metrics()